### Климентьева Наталья 24.01.2023

# SQL

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

### Задания

- Посчитайте, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитайте количество обзоров и среднюю оценку;
- Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
- Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
- Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.


### Цели исследования:

- Исследовать реляционную базу данных - содержание таблиц, их взаимосвязи;
- Выявить основные тренды в публицистике - пользовательские оценки, популярные издания и авторы, пользовательские обзоры. 

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

In [2]:
query = "SELECT * FROM books LIMIT 3"
pd.io.sql.read_sql(query, con = engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135


In [3]:
query = "SELECT * FROM authors LIMIT 3"
pd.io.sql.read_sql(query, con = engine)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie


In [4]:
query = "SELECT * FROM ratings LIMIT 3"
pd.io.sql.read_sql(query, con = engine)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5


In [5]:
query = "SELECT * FROM reviews LIMIT 3"
pd.io.sql.read_sql(query, con = engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...


In [6]:
query = "SELECT * FROM publishers LIMIT 3"
pd.io.sql.read_sql(query, con = engine)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books


- **Посчитайте, сколько книг вышло после 1 января 2000 года:**

In [7]:
query = '''
SELECT COUNT(DISTINCT book_id) количество_книг_после_1_01_2000
FROM books
WHERE publication_date::DATE >= '2000-01-01';
'''
pd.io.sql.read_sql(query, con = engine)

,количество_книг_после_1_01_2000
0,821


**Ответ:** 

После 1 января 2000 года вышла 821 книга.

- **Для каждой книги посчитайте количество обзоров и среднюю оценку:**

In [8]:
query = '''
WITH
a AS (SELECT book_id,
             ROUND(AVG(rating), 2) avg_rating
      FROM ratings
      GROUP BY book_id),
b AS (SELECT book_id,
             COUNT(DISTINCT review_id) count_review
      FROM reviews
      GROUP BY book_id)

SELECT title название_книги,
       COALESCE(b.count_review, 0)::int количество_обзоров,
       a.avg_rating средняя_оценка
FROM books c
LEFT JOIN b ON c.book_id=b.book_id
LEFT JOIN a ON c.book_id=a.book_id;
'''
pd.io.sql.read_sql(query, con = engine)

,название_книги,количество_обзоров,средняя_оценка
0,'Salem's Lot,2,3.67
1,1 000 Places to See Before You Die,1,2.50
2,13 Little Blue Envelopes (Little Blue Envelope...,3,4.67
3,1491: New Revelations of the Americas Before C...,2,4.50
4,1776,4,4.00
...,...,...,...
995,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.00
996,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.67
997,Essential Tales and Poems,0,4.00
998,Leonardo's Notebooks,0,4.00


**Ответ:**

Мы посчитали количество обзоров и среднюю оценку на каждую книгу. Для 6 книг обзоров не производилось.

- **Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры:**

In [9]:
query = '''
SELECT p.publisher издательство,
       COUNT(DISTINCT book_id) количество_книг
FROM books b
JOIN publishers p ON b.publisher_id=p.publisher_id
WHERE b.num_pages > 50
GROUP BY p.publisher_id
ORDER BY COUNT(DISTINCT book_id) DESC
LIMIT 1;
'''
pd.io.sql.read_sql(query, con = engine)

,издательство,количество_книг
0,Penguin Books,42


**Ответ:**

Издательство, выпустившее наибольшее число книг - Penguin Books. Всего под этим издательством вышло 42 книги.

- **Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками:**

Ниже код на автора с книгой с самой высокой средней оценкой (для сравнения).

In [14]:
query = '''
WITH
a AS (SELECT book_id,
             AVG(rating) avg_rating,
             COUNT(rating_id) count_rating
      FROM ratings
      GROUP BY book_id
      HAVING COUNT(rating_id) >= 50
      ORDER BY AVG(rating) DESC
      LIMIT 1)
      
      
SELECT au.author автор,
       ROUND(a.avg_rating, 4) средняя_оценка_книг
FROM books b
RIGHT JOIN a ON a.book_id=b.book_id
JOIN authors au ON au.author_id=b.author_id
'''
pd.io.sql.read_sql(query, con = engine)

,автор,средняя_оценка_книг
0,J.K. Rowling/Mary GrandPré,4.4146


Ниже код на поиск автора с самой высокой средней оценкой книг.

In [16]:
query = '''
SELECT au.author автор,
       ROUND(AVG(r.rating), 4) средняя_оценка_книг
FROM books b
LEFT JOIN authors au ON b.author_id=au.author_id
RIGHT JOIN ratings r ON b.book_id=r.book_id
WHERE b.book_id in (SELECT book_id
                    FROM ratings
                    GROUP BY book_id
                    HAVING COUNT(rating_id) >= 50)
GROUP BY au.author_id
ORDER BY ROUND(AVG(r.rating), 4) DESC
LIMIT 1
'''
pd.io.sql.read_sql(query, con = engine)

,автор,средняя_оценка_книг
0,J.K. Rowling/Mary GrandPré,4.2871


**Ответ:**

``J.K. Rowling/Mary GrandPré`` - автор с самой высокой средней оценкой его книг. Средняя оценка составляет ``4.2871``. Книги с менее 50 страниц исключены.

- **Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок:**

In [12]:
query = '''
WITH a AS (SELECT COUNT(review_id)
           FROM reviews
           WHERE username IN (SELECT username
                              FROM ratings
                              GROUP BY username
                              HAVING COUNT(rating_id) > 50)
           GROUP BY username)
SELECT ROUND(AVG(count), 0)::int среднее_количество_просмотров
FROM a
'''
pd.io.sql.read_sql(query, con = engine)

,среднее_количество_просмотров
0,24


**Ответ:**

Среднее количество просмотров от пользователей, которые поставили больше 50 оценок равно около 24.